# Fully-Connected Layers Tutorial on Fashion MNIST Data Set 

This is a naive tutorial on how to use `Dense` (Fully-connected Layer) to train and predict the Fashion MNIST data set

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.1.0


## Import the Fashion MNIST dataset

This guide uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels), as seen here:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST is intended as a drop-in replacement for the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc.) in a format identical to that of the articles of clothing you'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

Here, 60,000 images are used to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow. Import and load the Fashion MNIST data directly from TensorFlow:

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The *labels* are an array of integers, ranging from 0 to 9. These correspond to the *class* of clothing the image represents:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:
train_images = train_images.reshape((60000,28,28,1)) / 255.0
print("Train Images shape", train_images.shape)
test_images = test_images.reshape((10000,28,28,1)) /255.0

# train_labels = tf.one_hot(train_labels,10)
train_labels_h = tf.one_hot(train_labels,10) 
test_labels_h = tf.one_hot(test_labels,10)

Train Images shape (60000, 28, 28, 1)


To verify that the data is in the correct format and that you're ready to build and train the network, let's display the first 25 images from the *training set* and display the class name below each image.

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Set up the layers

The basic building block of a neural network is the *layer*. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [5]:
X_Input = keras.Input(shape=(28,28,1))
Xz1 = ZeroPadding2D((14,14))(X_Input)
Xz2 = ZeroPadding2D((28,28))(X_Input)
Xc = [Conv2D(3,(3,3),padding="same")(X_Input),
      Conv2D(4,(5,5), padding="same")(X_Input),
      Conv2D(10, (1,1), padding="same")(X_Input),
      MaxPool2D((2,2), padding="same")(Xz1),
      AveragePooling2D((3,3), padding="same")(Xz2)
      ]
X = Concatenate(axis=3)(Xc)
X = BatchNormalization(axis=3)(X)
X = Activation("relu")(X)
X = MaxPool2D((2,2))(X)
X

<tf.Tensor 'max_pooling2d_1/Identity:0' shape=(None, 14, 14, 19) dtype=float32>

In [6]:
Xz1 = ZeroPadding2D((7,7))(X)
Xz2 = ZeroPadding2D((14,14))(X)
Xc = [Conv2D(3,(3,3),padding="same")(X),
      Conv2D(4,(5,5), padding="same")(X),
      Conv2D(10, (1,1), padding="same")(X),
      MaxPool2D((2,2), padding="same")(Xz1),
      AveragePooling2D((3,3), padding="same")(Xz2)
      ]
X = Concatenate(axis=3)(Xc)
X = BatchNormalization(axis=3)(X)
X = Activation("relu")(X)
X = AveragePooling2D((3,3))(X)
X

<tf.Tensor 'average_pooling2d_2/Identity:0' shape=(None, 4, 4, 55) dtype=float32>

In [7]:
X = Flatten()(X)
X_Output = Dense(10, "softmax")(X)
X

<tf.Tensor 'flatten/Identity:0' shape=(None, 880) dtype=float32>

In [8]:
model = keras.Model(inputs=X_Input, outputs=X_Output)
model

In [9]:
optim = keras.optimizers.Adam(learning_rate=0.005)

In [10]:
model.compile(optimizer=optim,
              loss='categorical_crossentropy', metrics=['accuracy'])

### Feed the model

To start training,  call the `model.fit` method—so called because it "fits" the model to the training data:

In [11]:
model.fit(train_images, train_labels_h, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 847s 14ms/sample - loss: 0.4480 - accuracy: 0.8381
Epoch 2/10
60000/60000 [==============================] - 851s 14ms/sample - loss: 0.3239 - accuracy: 0.8831
Epoch 3/10
60000/60000 [==============================] - 852s 14ms/sample - loss: 0.2985 - accuracy: 0.8920
Epoch 4/10
60000/60000 [==============================] - 854s 14ms/sample - loss: 0.2816 - accuracy: 0.8976
Epoch 5/10
60000/60000 [==============================] - 849s 14ms/sample - loss: 0.2715 - accuracy: 0.9021
Epoch 6/10
60000/60000 [==============================] - 839s 14ms/sample - loss: 0.2642 - accuracy: 0.9051
Epoch 7/10
60000/60000 [==============================] - 841s 14ms/sample - loss: 0.2574 - accuracy: 0.9076
Epoch 8/10
60000/60000 [==============================] - 846s 14ms/sample - loss: 0.2528 - accuracy: 0.9087
Epoch 9/10
60000/60000 [==============================] - 848s 14ms/sample - loss: 0.2485 - accuracy: 0.9

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [13]:
train_loss, train_acc = model.evaluate(train_images,  train_labels_h, verbose=2)

print('\nTrain accuracy:', train_acc)

60000/60000 - 43s - loss: 0.2527 - accuracy: 0.9072

Train accuracy: 0.90725


In [14]:
test_loss, test_acc = model.evaluate(test_images,  test_labels_h, verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 8s - loss: 0.3050 - accuracy: 0.8899

Test accuracy: 0.8899
